In [1]:
import pandas as pd, numpy as np

%matplotlib inline

# DEAD ANIMALS

In [2]:
train = pd.read_csv('train.csv')
display(train.dtypes)
display(train.head())
display(train.describe())

AnimalID          object
Name              object
DateTime          object
OutcomeType       object
OutcomeSubtype    object
AnimalType        object
SexuponOutcome    object
AgeuponOutcome    object
Breed             object
Color             object
dtype: object

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan


,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
count,26729,19038,26729,26729,13117,26729,26728,26711,26729,26729
unique,26729,6374,22918,5,16,2,5,44,1380,366
top,A662227,Max,2015-08-11 00:00:00,Adoption,Partner,Dog,Neutered Male,1 year,Domestic Shorthair Mix,Black/White
freq,1,136,19,10769,7816,15595,9779,3969,8810,2824


In [3]:
import re
sterilized_pat = re.compile('.*(neutered|spayed).*', flags=re.IGNORECASE)

def purify_1(df):
    df = df.drop(['AnimalID'], axis=1)
    df['Name'] = pd.notnull(df['Name'])
    df['Sterilized'] = df.SexuponOutcome.str.match(sterilized_pat)
    df['Mix'] = df.Breed.str.contains('Mix')
    days_in_unit = {
    'day': 1,
    'days': 1,
    'week': 7,
    'weeks': 7,
    'month': 30,
    'months': 30,
    'years': 365,
    'year': 365
    }
    
    def strage_to_days(age):
        if not isinstance(age, str): return float('nan')
        age = age.split(' ')
        return int(age[0]) * days_in_unit[age[1]]
    df['AgeuponOutcome'] = df['AgeuponOutcome'].map(strage_to_days)

    return df

In [4]:
X = purify_1(train)
X

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,Sterilized,Mix
0,True,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,365.0,Shetland Sheepdog Mix,Brown/White,True,True
1,True,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,365.0,Domestic Shorthair Mix,Cream Tabby,True,True
2,True,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,730.0,Pit Bull Mix,Blue/White,True,True
3,False,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,21.0,Domestic Shorthair Mix,Blue Cream,False,True
4,False,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,730.0,Lhasa Apso/Miniature Poodle,Tan,True,False
5,True,2014-04-25 13:04:00,Transfer,Partner,Dog,Intact Female,30.0,Cairn Terrier/Chihuahua Shorthair,Black/Tan,False,False
6,True,2015-03-28 13:11:00,Transfer,Partner,Cat,Intact Male,21.0,Domestic Shorthair Mix,Blue Tabby,False,True
7,False,2015-04-30 17:02:00,Transfer,Partner,Cat,Unknown,21.0,Domestic Shorthair Mix,Brown Tabby,False,True
8,True,2014-02-04 17:17:00,Adoption,NaN,Dog,Spayed Female,150.0,American Pit Bull Terrier Mix,Red/White,True,True
9,False,2014-05-03 07:48:00,Adoption,Offsite,Dog,Spayed Female,365.0,Cairn Terrier,White,True,False
